# Benchmarking Cell Tracking Challenge Data

In [1]:
import os
from pathlib import Path
import pprint
import urllib.request
import zipfile

from tqdm import tqdm

from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatched, IOUMatched
from traccuracy.metrics import CTCMetrics, DivisionMetrics

pp = pprint.PrettyPrinter(indent=4)

In [ ]:
url = "http://data.celltrackingchallenge.net/training-datasets/Fluo-N2DL-HeLa.zip"
data_dir = 'downloads'

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

filename = url.split('/')[-1]
file_path = os.path.join(data_dir, filename)
ds_name = filename.split('.')[0]

In [ ]:
# Add a utility to make a progress bar when downloading the file
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

if not os.path.exists(file_path):
    print(f"Downloading {ds_name} data from the CTC website")
    # Downloading data
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, file_path, reporthook=t.update_to)
    # Unzip the data
    # TODO add a progress bar to zip as well
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

In [3]:

gt_data = load_ctc_data(
    '/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/Fluo-N2DL-HeLa/01_GT/TRA',
    '/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/Fluo-N2DL-HeLa/01_GT/TRA/man_track.txt'
)
pred_data = load_ctc_data(
    'sample-data/Fluo-N2DL-HeLa/01_RES', 
    'sample-data/Fluo-N2DL-HeLa/01_RES/res_track.txt'
)

Loading TIFFs: 100%|██████████| 92/92 [00:00<00:00, 158.07it/s]


Run CTC metrics with additional evaluation of division events.

In [4]:
ctc_results = run_metrics(
    gt_data=gt_data, 
    pred_data=pred_data, 
    matcher=CTCMatched, 
    metrics=[CTCMetrics, DivisionMetrics],
    metrics_kwargs=dict(
        frame_buffer=(0,1,2)
    )
)
pp.pprint(ctc_results)

Matching frames: 100%|██████████| 92/92 [00:20<00:00,  4.47it/s]


Matched 8600 out of 8639 ground truth nodes.
Matched 8600 out of 8600 predicted nodes.


Evaluating FP edges:  67%|██████▋   | 5683/8535 [00:12<00:06, 449.83it/s]


KeyboardInterrupt: 

Use an IOU matcher which supports a minimum threshold for overlap and run division metrics.

In [ ]:
iou_results = run_metrics(
    gt_data=gt_data, 
    pred_data=pred_data, 
    matcher=IOUMatched, 
    matcher_kwargs=dict(
        iou_threshold=0.5
    ),
    metrics=[DivisionMetrics], 
    metrics_kwargs=dict(
        frame_buffer=(0,1,2)
    )
)
pp.pprint(iou_results)